In [1]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.0


In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import numpy as np

batch_size = 64
num_classes = 10
epochs = 100
target_val_acc=0.9940

Using TensorFlow backend.


In [0]:
# input image dimensions.
img_rows, img_cols = 28, 28

# The data, split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [4]:
#Normalization.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('Original dataset:')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Original dataset:
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
#Generates augmented images from the x_train images 
#array (and uses corresponding y_train labels ofc).
datagen = ImageDataGenerator(rotation_range=20,
                            width_shift_range=0.10,
                            height_shift_range=0.10,
                            zoom_range=[0.8, 1.2],
                            shear_range=0.2)

train_gen = datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True)

#Appends augmented images to the original x_train and y_train arrays.

deg=1            #deg controls the extent of the augmentation. 1 indicates
                 #whole 60k images will be yielded in batches and used for
                 #training hence increasing train set to ~120k images.

print("Pre-processing images for data augmentation...Wait a minute...")
iterate=(int)((60000/batch_size)*deg)
for i in range(0,iterate):
    ret=train_gen.next()  
    x_train=np.append(x_train,ret[0],axis=0)
    y_train=np.append(y_train,ret[1])
print("Done adding!")
print("Total train images post data augmentation: ",x_train.shape[0])

Pre-processing images for data augmentation...Wait a minute...
Done adding!
Total train images post data augmentation:  119968


In [0]:
#Convert labels to one-hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def scheduler(epoch, lr):
  if epoch < 40:
    return 0.001
  else:
    return 0.001 * np.exp(0.1 * (40 - epoch))

#Callback for lr updation and stop training 
class myCallback(keras.callbacks.LearningRateScheduler):
    def __init__(self, schedule, val_target):
        super(myCallback, self).__init__(schedule)     
        self.val_target=val_target

    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_acc')>=self.val_target:
            self.model.stop_training = True
            print('Stopped training as target val_acc reached!')
        else:
            lr_=float(keras.backend.get_value(self.model.optimizer.lr))
            print('lr: ','{:.8f}'.format(lr_))    
    
callback0 = myCallback(scheduler,target_val_acc)

In [8]:
#Model
model = Sequential()

model.add(Conv2D(8, kernel_size=(3, 3),activation='relu', input_shape=input_shape))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(14, (3, 3),activation='relu'))
model.add(Conv2D(14, (3, 3), activation='relu'))
model.add(Conv2D(14, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

#Training
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[callback0])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 8)         584       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 8)         584       
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 22, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 8)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 9, 9, 14)          1022     

In [9]:
#validation Testing
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0224024444648996
Test accuracy: 0.9942
